In [1]:
%load_ext autoreload
%autoreload 2
from __future__ import division, print_function, absolute_import
import pickle
import os
import matplotlib.pyplot as plt
import sys
sys.path.append('..')
from edfreader import read_edf
import numpy as np
from lmfit import  Model, Parameters
%matplotlib inline

In [2]:
!ls ../data/

enregistrement_AM_2017-10-23_100057.asc
enregistrement_AM_2017-10-23_100057.edf
enregistrement_AM_2017-10-23_100057.pkl
enregistrement_BMC_2017-09-26_095637.asc
enregistrement_BMC_2017-09-26_095637.EDF
enregistrement_BMC_2017-09-26_095637.pkl
enregistrement_CS_2017-10-03_134421.asc
enregistrement_CS_2017-10-03_134421.edf
enregistrement_CS_2017-10-03_134421.pkl
enregistrement_DC_2017-09-27_161040.asc
enregistrement_DC_2017-09-27_161040.edf
enregistrement_DC_2017-09-27_161040.pkl
enregistrement_FM_2017-10-03_143803.asc
enregistrement_FM_2017-10-03_143803.EDF
enregistrement_FM_2017-10-03_143803.pkl
enregistrement_IP_2017-09-28_115250.asc
enregistrement_IP_2017-09-28_115250.edf
enregistrement_IP_2017-09-28_115250.pkl
enregistrement_LB_2017-09-20_151043 (1).asc
enregistrement_LB_2017-09-20_151043.asc
enregistrement_LB_2017-09-20_151043.edf
enregistrement_LB_2017-09-20_151043.pkl
enregistrement_OP_2017-10-26_121823.asc
enregistrement_OP_2017-10-26_121823.edf
enregistrement_OP_2017-10-26_1218

In [3]:
timeStr = '2017-11-08_150410'
observer = 'TN'
mode = 'enregistrement'

In [4]:
file = os.path.join('..','data', mode + '_' + observer + '_' + timeStr + '.pkl')
with open(file, 'rb') as fichier :
    exp = pickle.load(fichier, encoding='latin1')
      
resultats = os.path.join('..','data', mode + '_' + observer + '_' + timeStr + '.asc')
data = read_edf(resultats, 'TRIALID')

In [5]:
N_trials = exp['N_trials']
N_blocks = exp['N_blocks']
screen_width_px = exp['screen_width_px']
screen_height_px = exp['screen_height_px']
V_X = exp['V_X']
RashBass = exp['RashBass']
stim_tau = exp['stim_tau']
p = exp['p']

In [6]:
def exponentiel (x, tau, maxi, start_anti, v_anti, latence, bino) :
    '''
    tau -- courbe
    maxi -- maximum
    latence -- tps où commence le mvt
    bino -- binomial
    
    start_anti = debut de l'anticipation
    v_anti =  vitesse de l'anticipation
    ''' 
    v_anti = v_anti/1000 # pour passer de sec à ms
    time = np.arange(len(x))
    vitesse = []
                
    for t in range(len(time)):
        
        if start_anti >= latence :
            if time[t] < latence :
                vitesse.append(0)
            else :
                vitesse.append((bino*2-1)*maxi*(1-np.exp(-1/tau*(time[t]-latence))))
        else :

            if time[t] < start_anti :
                vitesse.append(0)
            else :
                if time[t] < latence :
                    #vitesse.append((bino*2-1)*(time[t]-start_anti)*v_anti)
                    vitesse.append((time[t]-start_anti)*v_anti)
                    x = (time[t]-start_anti)*v_anti
                else :
                    vitesse.append((bino*2-1)*maxi*(1-np.exp(-1/tau*(time[t]-latence)))+x)
    return vitesse

In [7]:
plt.plot(exponentiel(np.arange(2000), tau=15, maxi=15., start_anti=300, latence=1000., v_anti=0.6, bino=0), c='r')

[[Model]]
    Model(exponentiel)
[[Fit Statistics]]
    # function evals   = 148
    # data points      = 1724
    # variables        = 5
    chi-square         = 54349.801
    reduced chi-square = 31.617
    Akaike info crit   = 5959.168
    Bayesian info crit = 5986.430
[[Variables]]
    tau:          59.7507082 +/- 34.19621 (57.23%) (init= 15)
    maxi:         8.55014968 +/- 1.113515 (13.02%) (init= 15)
    latence:      1234.00001 +/- 41.04071 (3.33%) (init= 1284)
    start_anti:   1095.01507 +/- 24.32906 (2.22%) (init= 1084)
    v_anti:       39.9998868 +/- 12.82564 (32.06%) (init= 0)
    bino:         1 (fixed)
[[Correlations]] (unreported correlations are <  0.100)
    C(tau, latence)              = -0.908 
    C(start_anti, v_anti)        =  0.864 
    C(maxi, v_anti)              = -0.848 
    C(maxi, start_anti)          = -0.512 
    C(maxi, latence)             = -0.269 
    C(latence, v_anti)           =  0.160 
    C(tau, maxi)                 =  0.147 



In [8]:
fig_width= 15
fig, axs = plt.subplots(1, 1, figsize=(fig_width, (fig_width/2)/1.6180))
block = 0
trial = 4

trial_data = trial + N_trials*block

data_x = data[trial_data]['x']
data_y = data[trial_data]['y']
trackertime = data[trial_data]['trackertime']

StimulusOn = data[trial_data]['events']['msg'][10][0]
StimulusOf = data[trial_data]['events']['msg'][14][0]
TargetOn = data[trial_data]['events']['msg'][15][0]
TargetOff = data[trial_data]['events']['msg'][16][0]
saccades = data[trial_data]['events']['Esac']
trackertime_0 = data[trial_data]['trackertime'][0]

gradient_x = np.gradient(data_x) # gradient en px/ms
gradient_deg = gradient_x * 1/exp['px_per_deg'] * 1000 # gradient en deg/sec

##################################################
# SUPPRESSION DES SACCADES
##################################################
gradient_deg_NAN = gradient_deg

for s in range(len(saccades)) :
    if saccades[s][1]-trackertime_0+15 <= (len(trackertime)) :
        for x_data in np.arange((saccades[s][0]-trackertime_0-5), (saccades[s][1]-trackertime_0+15)) :
            gradient_deg_NAN[x_data] = np.nan
    else :
        for x_data in np.arange((saccades[s][0]-trackertime_0-5), (len(trackertime))) :
            gradient_deg_NAN[x_data] = np.nan

stop_latence = []    
for s in range(len(saccades)) :
    if (saccades[s][0]-trackertime_0) >= (TargetOn-trackertime_0+100) :
        stop_latence.append((saccades[s][0]-trackertime_0))
if stop_latence==[] :
    stop_latence.append(len(trackertime))
##################################################

start = TargetOn

StimulusOn_s = StimulusOn - start
StimulusOf_s = StimulusOf - start
TargetOn_s = TargetOn - start
TargetOff_s = TargetOff - start
trackertime_s = trackertime - start

# FIT
model = Model(exponentiel)
bino=p[trial, block, 0]
params = Parameters()

params.add('tau', value=15., min=13., max=80.)#, vary=False)
params.add('maxi', value=15., min=1., max=40.)#, vary=False)
params.add('latence', value=TargetOn-trackertime_0+100, min=TargetOn-trackertime_0+50, max=stop_latence[0])
params.add('start_anti', value=TargetOn-trackertime_0-100, min=StimulusOf-trackertime_0, max=TargetOn-trackertime_0-50)
params.add('v_anti', value=(bino*2-1)*0, min=-40., max=40.)
params.add('bino', value=bino, min=0, max=1, vary=False)

#result_deg = model.fit(new_gradient_deg, params, x=new_time)
result_deg = model.fit(gradient_deg_NAN[:-250], params, x=trackertime[:-250], fit_kws={'nan_policy': 'omit'})

debut  = TargetOn - trackertime_0 # TargetOn - temps_0

axs.axis([StimulusOn_s-10, TargetOff_s+10, -40, 40])

axs.plot(trackertime_s, gradient_deg_NAN, color='k', alpha=0.6)
axs.plot(trackertime_s[:-250], result_deg.init_fit, 'r--', linewidth=2)
axs.plot(trackertime_s[:-250], result_deg.best_fit, color='r', linewidth=2)
axs.plot(trackertime_s, np.ones(np.shape(trackertime_s)[0])*(bino*2-1)*15, color='k', linewidth=0.2, alpha=0.2)
axs.plot(trackertime_s, np.ones(np.shape(trackertime_s)[0])*(bino*2-1)*10, color='k', linewidth=0.2, alpha=0.2)


axs.axvspan(StimulusOn_s, StimulusOf_s, color='k', alpha=0.2)
axs.axvspan(StimulusOf_s, TargetOn_s, color='r', alpha=0.2)
axs.axvspan(TargetOn_s, TargetOff_s, color='k', alpha=0.15)
for s in range(len(saccades)) :
    axs.axvspan(saccades[s][0]-start, saccades[s][1]-start, color='k', alpha=0.2)

start_anti = result_deg.values['start_anti']-debut
v_anti = result_deg.values['v_anti']
latence = result_deg.values['latence']-debut
tau = result_deg.values['tau']
maxi = result_deg.values['maxi']

axs.bar(latence, 80, bottom=-40, color='r', width=6, linewidth=0)

axs.text(StimulusOn_s+(StimulusOf_s-StimulusOn_s)/2, 31, "FIXATION", color='k', fontsize=16, ha='center', va='bottom')
axs.text(StimulusOf_s+(TargetOn_s-StimulusOf_s)/2, 31, "GAP", color='r', fontsize=16, ha='center', va='bottom')
axs.text(TargetOn_s+(TargetOff_s-TargetOn_s)/2, 31, "POURSUITE", color='k', fontsize=16, ha='center', va='bottom')
axs.text(latence+25, -35, "Latence", color='r', fontsize=14)#,  weight='bold')
axs.text(StimulusOn_s+15, 18, "start_anti: %s \nv_anti: %s"%(start_anti, v_anti), color='k', fontsize=14, va='bottom')
axs.text(StimulusOn_s+15, -18, "latence: %s \ntau: %s \nmaxi: %s"%(latence, tau, maxi), color='k', fontsize=14, va='top')

axs.set_xlabel('Temps (ms)', fontsize=12)
axs.set_ylabel('Vitesse', fontsize=12)
plt.show()
print(result_deg.fit_report())

In [9]:
liste_start_anti = []
liste_liste_v_anti = []
liste_latence = []
liste_tau = []
liste_maxi = []
liste_mean = []

for block in range(N_blocks) :
    fig_width= 12
    fig, axs = plt.subplots(N_trials, 1, figsize=(fig_width, (fig_width*(N_trials/2))/1.6180))

    block_start_anti = []
    block_liste_v_anti = []
    block_latence = []
    block_tau = []
    block_maxi = []
    block_mean = []

    for trial in range(N_trials) :

        print(block, trial)
        
        trial_data = trial + N_trials*block
        data_x = data[trial_data]['x']
        data_y = data[trial_data]['y']
        trackertime = data[trial_data]['trackertime']

        StimulusOn = data[trial_data]['events']['msg'][10][0]
        StimulusOf = data[trial_data]['events']['msg'][14][0]
        TargetOn = data[trial_data]['events']['msg'][15][0]
        TargetOff = data[trial_data]['events']['msg'][16][0]
        saccades = data[trial_data]['events']['Esac']

        trackertime_0 = data[trial_data]['trackertime'][0]

        gradient_x = np.gradient(data_x)
        gradient_deg = gradient_x * 1/exp['px_per_deg'] * 1000 # gradient en deg/sec

        ##################################################
        # SUPPRESSION DES SACCADES
        ##################################################
        gradient_deg_NAN = gradient_deg
        for s in range(len(saccades)) :
            if saccades[s][1]-trackertime_0+15 <= (len(trackertime)) :
                for x_data in np.arange((saccades[s][0]-trackertime_0-5), (saccades[s][1]-trackertime_0+15)) :
                    gradient_deg_NAN[x_data] = np.nan
            else :
                for x_data in np.arange((saccades[s][0]-trackertime_0-5), (len(trackertime))) :
                    gradient_deg_NAN[x_data] = np.nan
            
        stop_latence = []    
        for s in range(len(saccades)) :
            if (saccades[s][0]-trackertime_0) >= (TargetOn-trackertime_0+100) :
                stop_latence.append((saccades[s][0]-trackertime_0))
        if stop_latence==[] :
            stop_latence.append(len(trackertime))
        ##################################################
        
        start = TargetOn
        
        StimulusOn_s = StimulusOn - start
        StimulusOf_s = StimulusOf - start
        TargetOn_s = TargetOn - start
        TargetOff_s = TargetOff - start
        trackertime_s = trackertime - start
        
        ##################################################
        # FIT
        ##################################################
        model = Model(exponentiel)
        bino=p[trial, block, 0]
        params = Parameters()
        params.add('tau', value=15., min=13., max=70.)#, vary=False)
        params.add('maxi', value=15., min=1., max=40.)#, vary=False)
        params.add('latence', value=TargetOn-trackertime_0+100, min=TargetOn-trackertime_0+50, max=stop_latence[0])
        params.add('start_anti', value=TargetOn-trackertime_0-100, min=StimulusOf-trackertime_0, max=TargetOn-trackertime_0-50) #min=StimulusOf-trackertime_0+100, max=TargetOn-trackertime_0-50
        params.add('v_anti', value=0., min=-40., max=40.)
        params.add('bino', value=bino, min=0, max=1, vary=False)

        result_deg = model.fit(gradient_deg_NAN[:-250], params, x=trackertime[:-250], fit_kws={'nan_policy': 'omit'})
        ##################################################

        axs[trial].cla() # pour remettre ax figure a zero
        axs[trial].axis([StimulusOn_s-10, TargetOff_s+10, -40, 40])

        axs[trial].plot(trackertime_s, gradient_deg_NAN, color='k', alpha=0.6)
        axs[trial].plot(trackertime_s[:-250], result_deg.init_fit, 'r--', linewidth=2)
        axs[trial].plot(trackertime_s[:-250], result_deg.best_fit, color='r', linewidth=2)
        axs[trial].plot(trackertime_s, np.ones(np.shape(trackertime_s)[0])*(bino*2-1)*(15), color='k', linewidth=0.2, alpha=0.2)
        axs[trial].plot(trackertime_s, np.ones(np.shape(trackertime_s)[0])*(bino*2-1)*(10), color='k', linewidth=0.2, alpha=0.2)
        axs[trial].axvspan(StimulusOn_s, StimulusOf_s, color='k', alpha=0.2)
        axs[trial].axvspan(StimulusOf_s, TargetOn_s, color='r', alpha=0.2)
        axs[trial].axvspan(TargetOn_s, TargetOff_s, color='k', alpha=0.15)
        for s in range(len(saccades)) :
            axs[trial].axvspan(saccades[s][0]-start, saccades[s][1]-start, color='k', alpha=0.2)

        debut  = TargetOn - trackertime_0 # TargetOn - temps_0
        
        start_anti = result_deg.values['start_anti']-debut
        v_anti = result_deg.values['v_anti']
        latence = result_deg.values['latence']-debut
        tau = result_deg.values['tau']
        maxi = result_deg.values['maxi']

        block_start_anti.append(start_anti)
        block_liste_v_anti.append(v_anti)
        block_latence.append(latence)
        block_tau.append(tau)
        block_maxi.append(maxi)
        block_mean.append(np.nanmean(gradient_deg_NAN[debut-50:debut+50]))

        axs[trial].bar(latence, 80, bottom=-40, color='r', width=6, linewidth=0)
        
        if trial==0 :
            axs[trial].text(StimulusOn_s+(StimulusOf_s-StimulusOn_s)/2, 31, "FIXATION", color='k', fontsize=16, ha='center', va='bottom')
            axs[trial].text(StimulusOf_s+(TargetOn_s-StimulusOf_s)/2, 31, "GAP", color='r', fontsize=16, ha='center', va='bottom')
            axs[trial].text(TargetOn_s+(TargetOff_s-TargetOn_s)/2, 31, "POURSUITE", color='k', fontsize=16, ha='center', va='bottom')
            axs[trial].text(latence+25, -35, "Latence"%(latence), color='r', fontsize=14)#,  weight='bold')
        #axs[trial].text(StimulusOn+15, -2, "%s"%(result.fit_report()), color='k', fontsize=15)
        axs[trial].text(StimulusOn_s+15, 18, "start_anti: %s \nv_anti: %s"%(start_anti, v_anti), color='k', fontsize=14, va='bottom')
        axs[trial].text(StimulusOn_s+15, -18, "latence: %s \ntau: %s \nmaxi: %s"%(latence, tau, maxi), color='k', fontsize=14, va='top')

        axs[trial].set_xlabel('Time (ms)', fontsize=9)
        axs[trial].set_ylabel(trial+1, fontsize=9)

    liste_start_anti.append(block_start_anti)
    liste_liste_v_anti.append(block_liste_v_anti)
    liste_latence.append(block_latence)
    liste_tau.append(block_tau)
    liste_maxi.append(block_maxi)
    liste_mean.append(block_mean)

    plt.tight_layout() # pour supprimer les marge trop grande
    plt.subplots_adjust(hspace=0) # pour enlever espace entre les figures

    plt.savefig('figures/Test_Fit_%s_%s_block-%s.pdf'%(observer, timeStr, block+1))

plt.close()

param = {}
param['observer'] = observer
param['start_anti'] = liste_start_anti
param['v_anti'] = liste_liste_v_anti
param['latence'] = liste_latence
param['tau'] = liste_tau
param['maxi'] = liste_maxi
param['moyenne'] = liste_mean

file = os.path.join('parametre', 'Test_' + observer + '_param_Fit.pkl')
with open(file, 'wb') as fichier:
    f = pickle.Pickler(fichier)
    f.dump(param)

print('FIN !!!')

0 0
0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8
0 9
0 10
0 11
0 12
0 13
0 14
0 15
0 16
0 17
0 18
0 19
0 20
0 21
0 22
0 23
0 24
0 25
0 26
0 27
0 28
0 29
0 30
0 31
0 32
0 33
0 34
0 35
0 36
0 37
0 38
0 39
0 40
0 41
0 42
0 43
0 44
0 45
0 46
0 47
0 48
0 49
0 50
0 51
0 52
0 53
0 54
0 55
0 56
0 57
0 58
0 59
0 60
0 61
0 62
0 63
0 64
0 65
0 66
0 67
0 68
0 69
0 70
0 71
0 72
0 73
0 74
0 75
0 76
0 77
0 78
0 79
0 80
0 81
0 82
0 83
0 84
0 85
0 86
0 87
0 88
0 89
0 90
0 91
0 92
0 93
0 94
0 95
0 96
0 97
0 98
0 99
0 100
0 101
0 102
0 103
0 104
0 105
0 106
0 107
0 108
0 109
0 110
0 111
0 112
0 113
0 114
0 115
0 116
0 117
0 118
0 119
0 120
0 121
0 122
0 123
0 124
0 125
0 126
0 127
0 128
0 129
0 130
0 131
0 132
0 133
0 134
0 135
0 136
0 137
0 138
0 139
0 140
0 141
0 142
0 143
0 144
0 145
0 146
0 147
0 148
0 149
0 150
0 151
0 152
0 153
0 154
0 155
0 156
0 157
0 158
0 159
0 160
0 161
0 162
0 163
0 164
0 165
0 166
0 167
0 168
0 169
0 170
0 171
0 172
0 173
0 174
0 175
0 176
0 177
0 178
0 179
0 180
0 181
0 182
0 183
0 184


/home/drone/.local/lib/python3.5/site-packages/ipykernel_launcher.py:110: RuntimeWarning: Mean of empty slice


2 98
2 99
2 100
2 101
2 102
2 103
2 104
2 105
2 106
2 107
2 108
2 109
2 110
2 111
2 112
2 113
2 114
2 115
2 116
2 117
2 118
2 119
2 120
2 121
2 122
2 123
2 124
2 125
2 126
2 127
2 128
2 129
2 130
2 131
2 132
2 133
2 134
2 135
2 136
2 137
2 138
2 139
2 140
2 141
2 142
2 143
2 144
2 145
2 146
2 147
2 148
2 149
2 150
2 151
2 152
2 153
2 154
2 155
2 156
2 157
2 158
2 159
2 160
2 161
2 162
2 163
2 164
2 165
2 166
2 167
2 168
2 169
2 170
2 171
2 172
2 173
2 174
2 175
2 176
2 177
2 178
2 179
2 180
2 181
2 182
2 183
2 184
2 185
2 186
2 187
2 188
2 189
2 190
2 191
2 192
2 193
2 194
2 195
2 196
2 197
2 198
2 199
FIN !!!


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.
